In [1]:
import numpy
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer,IDF
from pyspark.ml.classification import NaiveBayes
import pandas as pd
from pyspark.sql.functions import col, expr, when
from pyspark.sql.types import DoubleType
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sklearn.metrics import confusion_matrix

In [2]:
#AFR_df = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load("/FileStore/tables/sample.csv")
AFR_df = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load("/FileStore/tables/Reviews.csv")

AFR_df = AFR_df.na.drop()
#AFR_df=sqlContext.createDataFrame(df)

data_frame= AFR_df.withColumn("label", when(AFR_df.Score > 3, 1).otherwise(0))
# #Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and lr.

changedTypedf = data_frame.withColumn("label", data_frame["label"].cast(DoubleType()))

In [3]:
X = changedTypedf['Summary','label']

In [4]:
train, test = X.randomSplit([0.7, 0.3], seed = 2018)

In [5]:
tokenizer = Tokenizer(inputCol="Summary", outputCol="words")
#wordsData = tokenizer.transform(train)

hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="rawFeatures", numFeatures=20)
#featurizedData = hashingTF.transform(wordsData)

idf = IDF(inputCol=hashingTF.getOutputCol(), outputCol="features")
#idfModel = idf.fit(featurizedData)
#rescaledData = idfModel.transform(featurizedData)

In [6]:
lr = LogisticRegression(maxIter=10, regParam=0.001)

#lr = NaiveBayes(smoothing=1)

pipeline = Pipeline(stages=[tokenizer, hashingTF,idf, lr])

# # # # Fit the pipeline to training documents.
model = pipeline.fit(train)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-712296903567603> in <module> () 
 6 
 7 # # # # Fit the pipeline to training documents. 
 ----> 8 model = pipeline . fit ( train ) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/pipeline.py in _fit (self, dataset) 
 107 dataset = stage . transform ( dataset ) 
 108 else : # must be an Estimator 
 --> 109 model = stage . fit ( dataset ) 
 110 transformers . append ( model ) 
 111 if i < indexOfLastEstimator : 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit (self, dataset) 
 293 
 294 def _fit ( self , dataset ) : 
 --> 295 java_model = self . _fit_java ( dataset ) 
 296 model = self . _create_model ( java_model ) 
 297 return self . _copyValues ( model ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit_java (self, dataset) 
 290 """
 291 self . _transfer_params_to_java ( ) 
 --> 292 return self . _java_obj . fit ( dataset . _jdf ) 
 293 
 294 def _fit ( self , dataset ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o383.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 4.0 failed 1 times, most recent failure: Lost task 7.0 in stage 4.0 (TID 25, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$createTransformFunc$1: (string) => array<string>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:634)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$class.foreach(Iterator.scala:893)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1336)
	at scala.collection.TraversableOnce$class.foldLeft(TraversableOnce.scala:157)
	at scala.collection.AbstractIterator.foldLeft(Iterator.scala:1336)
	at scala.collection.TraversableOnce$class.aggregate(TraversableOnce.scala:214)
	at scala.collection.AbstractIterator.aggregate(Iterator.scala:1336)
	at org.apache.spark.rdd.RDD$$anonfun$treeAggregate$1$$anonfun$24.apply(RDD.scala:1161)
	at org.apache.spark.rdd.RDD$$anonfun$treeAggregate$1$$anonfun$24.apply(RDD.scala:1161)
	at org.apache.spark.rdd.

In [7]:


prediction = model.transform(test)
selected = prediction.select("prediction")



In [8]:
# for row in selected.collect():
#     summary, prob, prediction = row
#     print("(%s) --> prob=%s, prediction=%f" % (summary, str(prob), prediction))

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)

y_true = test.select("label")
y_true = y_true.toPandas()

y_pred = prediction.select("prediction")
y_pred = y_pred.toPandas()

classes = [0,1]

cnf_matrix = confusion_matrix(y_true, y_pred,labels=classes)

print("Accuracy = %g" % (accuracy))
print(cnf_matrix)

Accuracy = 0.755643
[[ 45 705]
 [ 42 2265]]